## Creating and Using a Diabetes Prediction Model Leveraging Tensorflow and Virtuoso

Creating a Tensorflow prediction model from data sourced directly from Virtuoso, using PyODBC. This example uses PyODBC with SQL and SPASQL (SPARQL-within-SQL) for retrieving data that’s applied to the prediction model.

The [dataset](https://www.kaggle.com/mathchi/diabetes-data-set) contains data about female patients of Pima Indian heritage, that are at least 21 years old.

Columns:

* Pregnancies: Number of times pregnant
* Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
* BloodPressure: Diastolic blood pressure (mm Hg)
* SkinThickness: Triceps skin fold thickness (mm)
* Insulin: 2-Hour serum insulin (mu U/ml)
* BMI: Body mass index (weight in kg/(height in m)^2)
* DiabetesPedigreeFunction: Diabetes pedigree function
* Age: Age (years)
* Outcome: Class variable (0 or 1)

## Import Required Libraries

In [814]:
import pandas as pd
import pyodbc
import tensorflow as tf
import datetime
import numpy as np

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

### Set Connection, and Create Dataframes

In [833]:
cnxn = pyodbc.connect("DSN=Local Virtuoso;UID=dba;pwd=dba")

q = 'SELECT * FROM "tensorflow"."diabetes".data'

d = pd.read_sql_query(q,cnxn)

d.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [816]:
df = pd.DataFrame(d)

outcome = df.pop("Outcome")

df.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [817]:
feat_names = df.columns.tolist()
feats = df[feat_names]


In [818]:
print(feats.shape)

(767, 8)


### Convert to Tensor and continue in TensorFlow

In [819]:
print(tf.convert_to_tensor(feats))

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(feats)
normalizer(feats.iloc[:3])

tf.Tensor(
[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  2.    122.     70.    ...  36.8     0.34   27.   ]
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]], shape=(767, 8), dtype=float64)


<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.6387271 ,  0.84705454,  0.14960383,  0.90778947, -0.6935593 ,
         0.20362139,  0.4676379 ,  1.4246367 ],
       [-0.8458293 , -1.1243613 , -0.16038112,  0.5315605 , -0.6935593 ,
        -0.6842581 , -0.36549374, -0.1917781 ],
       [ 1.2325497 ,  1.9422855 , -0.26370946, -1.2868794 , -0.6935593 ,
        -1.1028302 ,  0.6034746 , -0.10670363]], dtype=float32)>

### Create Model

In [820]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(50, activation='relu',input_shape=(767,8)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
  ])

  model.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

model = get_basic_model()

model.fit(feats, outcome, epochs=50, batch_size=BATCH_SIZE)

Epoch 1/50
384/384 [==============================] - 2s 710us/step - loss: 0.7440 - accuracy: 0.7027
Epoch 2/50
384/384 [==============================] - 0s 661us/step - loss: 0.4886 - accuracy: 0.7627
Epoch 3/50
384/384 [==============================] - 0s 712us/step - loss: 0.4657 - accuracy: 0.7757
Epoch 4/50
384/384 [==============================] - 0s 671us/step - loss: 0.4539 - accuracy: 0.7927
Epoch 5/50
384/384 [==============================] - 0s 669us/step - loss: 0.4466 - accuracy: 0.7797
Epoch 6/50
384/384 [==============================] - 0s 662us/step - loss: 0.4321 - accuracy: 0.7784
Epoch 7/50
384/384 [==============================] - 0s 681us/step - loss: 0.4272 - accuracy: 0.7888
Epoch 8/50
384/384 [==============================] - 0s 687us/step - loss: 0.4243 - accuracy: 0.7927
Epoch 9/50
384/384 [==============================] - 0s 681us/step - loss: 0.4224 - accuracy: 0.8031
Epoch 10/50
384/384 [==============================] - 0s 688us/step - loss: 0.415

In [821]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


## Test Predictions

In [822]:
sample = [[6,148,72,35,0,33.6,.627,50],[6,148,72,35,0,33.6,.627,60],[1,85,66,29,0,26.6,0.351,55],[5,80,100,29,0,30.6,.422,55]]
prediction = model.predict(sample)
print(prediction)

[[4.4903778e-03 9.9550927e-01 4.6293195e-08 4.7416052e-08 7.8958088e-08
  3.9591608e-08 3.0249939e-08 4.9269641e-08 4.2966860e-08 5.1498901e-08]
 [2.5909062e-02 9.7409093e-01 6.1373875e-09 7.8961948e-09 1.0615281e-08
  6.2274177e-09 3.9675867e-09 8.6518686e-09 5.0128413e-09 7.6287554e-09]
 [9.7235501e-01 2.7645022e-02 1.1435567e-12 1.8749596e-12 1.1209251e-12
  2.1541915e-12 3.4430581e-12 4.3712850e-12 1.1047766e-12 1.2308910e-12]
 [8.8945156e-01 1.1054842e-01 7.2743711e-14 1.2646950e-13 1.0237515e-13
  1.5663017e-13 1.8216020e-13 3.4198976e-13 6.0707353e-14 5.6359174e-14]]


### Show Predicted Classes

1 = Diabetic <br>
0 = Not Diabetic

In [823]:
classes = np.argmax(prediction, axis = 1)
print(classes)

[1 1 0 0]


## Option 1: Adding New Entries And Predictions Using SPARQL INSERT

The register_and_predict() function inserts recorded data from parameters, and predicted data from TensorFlow,
into Virtuoso as RDF triples, via SPARQL INSERT 

### Create Function for Inserting Data

In [826]:
cursor = cnxn.cursor()
def register_and_predict(id,pregnancies, glucose, bp, st, insulin, bmi, dbf, age):
        
        #Get Preditcion Data
        data = [[pregnancies,glucose,bp,st,insulin,bmi,dbf,age]]
        x = model.predict(data)
        y = np.argmax(x, axis = 1)
        
        #Insert Patient Data and Prediction
        q1 = '''
             SPARQL \n 
             PREFIX : <#>\n 
             INSERT INTO GRAPH <urn:diabetes:data:test> \n 
             {\n \
               <#%s> :pregnancies '%d'^^xsd:integer; \n 
               :glucose '%d'^^xsd:integer; \n 
               :bloodPressure '%d'^^xsd:integer;\n 
               :skinThickness '%d'^^xsd:integer; \n
               :insulin '%d'^^xsd:integer; \n
               :bmi '%d'^^xsd:integer; \n 
               :diabetesPedigreeFunction '%d'^^xsd:integer; \n  
               :age '%d'^^xsd:integer; \n
               :hasDiabetes '%d'^^xsd:boolean;
               :time ?time. \n
             }\n
             WHERE\n
             {\n
                  BIND(xsd:dateTime(now()) as ?time).\n
             }\n
             '''%(id, pregnancies, glucose, bp, st, insulin, bmi, dbf, age,y)
        
        #Execute Query
        cursor.execute(q1)
        print( "Added Patient: <#%s>, %s, %s" % ( id,data[0], bool(y) ) )        

### Add Entries and Predictions to Named Graph in Virtuoso Instance

In [827]:
register_and_predict('12345',6,148,72,35,0,33.6,.627,50)
register_and_predict('1a2b3',2,108,84,35,0,33.6,0,30)
register_and_predict('xxyyzz',0,28,72,35,0,33.6,.627,40)

Added Patient: <#12345>, [6, 148, 72, 35, 0, 33.6, 0.627, 50], True
Added Patient: <#1a2b3>, [2, 108, 84, 35, 0, 33.6, 0, 30], False
Added Patient: <#xxyyzz>, [0, 28, 72, 35, 0, 33.6, 0.627, 40], True


### Query the targeted Named Graph, and confirm new entries

In [828]:
q = """
    SPARQL \n
    PREFIX : <#> \n
    SELECT * \n
    FROM <urn:diabetes:data:test> \n
    WHERE \n
        {\n
            ?patient :pregnancies ?pregnancies;
            :glucose ?glucose;
            :bloodPressure ?bloodPressure;
            :skinThickness ?skinThickness;
            :insulin ?insulin;
            :bmi ?bmi;
            :diabetesPedigreeFunction ?diabetesPedigreeFunction;
            :hasDiabetes ?diabetesPrediction;
            :time ?testEntryTime.
        }
    """

sparql_res = pd.read_sql_query(q,cnxn)
sparql_res.head()

,patient,pregnancies,glucose,bloodPressure,skinThickness,insulin,bmi,diabetesPedigreeFunction,diabetesPrediction,testEntryTime
0,# 1 2 3 4 5 ,6 ,1 4 8 ,7 2 ,3 5 ,0 ,3 3 ,0 ,1 ,2 0 2 1 - 1 2 - 0 6   1 9 : 5 4 : 3 5 . 8 3 1 ...
1,# 1 a 2 b 3 ,2 ,1 0 8 ,8 4 ,3 5 ,0 ,3 3 ,0 ,0 ,2 0 2 1 - 1 2 - 0 6   1 9 : 5 4 : 3 5 . 8 3 1 ...
2,# x x y y z z ,0 ,2 8 ,7 2 ,3 5 ,0 ,3 3 ,0 ,1 ,2 0 2 1 - 1 2 - 0 6   1 9 : 5 4 : 3 5 . 8 3 1 ...


## Option 2: Adding New Patient Data + Predictions Using SQL INSERT

The register_and_predict_sql() function inserts recorded data from parameters, and predicted data from TensorFlow,
into a designated SQL table, via SQL INSERT 

### Create Function for Inserting Data

In [835]:
cnxn.setencoding(encoding='utf-8')
cursor = cnxn.cursor()
def register_and_predict_sql(id,pregnancies, glucose, bp, st, insulin, bmi, dbf, age):
    #Get Preditcion Data
    data = [[pregnancies,glucose,bp,st,insulin,bmi,dbf,age]]
    x = model.predict(data)
    y = np.argmax(x, axis = 1)
        
    #Insert Patient Data and Prediction
    q = """
            INSERT INTO \n
                "tensorflow"."diabetes"."test_entry" 
            VALUES('%s',%d,%d,%d,%d,%d,%d,%d,%d,%d, curdatetime())""" % (id, pregnancies, glucose, bp, st, insulin, bmi, dbf, age,y)
    cursor.execute(q)
    print( "Added Patient: <#%s>, %s, %s" % ( id,data[0], bool(y) ) )

### Add Entries And Predictions to a Table in Virtuoso Instance

In [836]:
register_and_predict_sql('12345',6,148,72,35,0,33.6,.627,50)
register_and_predict_sql('1a2b3',2,108,84,35,0,33.6,0,30)
register_and_predict_sql('xxyyzz',0,28,72,35,0,33.6,.627,40)

Added Patient: <#12345>, [6, 148, 72, 35, 0, 33.6, 0.627, 50], True
Added Patient: <#1a2b3>, [2, 108, 84, 35, 0, 33.6, 0, 30], False
Added Patient: <#xxyyzz>, [0, 28, 72, 35, 0, 33.6, 0.627, 40], True


### Query the targeted table, and confirm new entries

In [837]:
sql_res = pd.read_sql_query("""SELECT * FROM "tensorflow"."diabetes"."test_entry" """,cnxn)
sql_res.head()

,patient_id,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetic_pedigree_function,age,prediction,test_entry_time
0,1 2 3 4 5 ,6,148,72,35,0,33.0,0.0,50,1,2021-12-06 19:56:44.666191
1,1 a 2 b 3 ,2,108,84,35,0,33.0,0.0,30,0,2021-12-06 19:56:44.708945
2,x x y y z z ,0,28,72,35,0,33.0,0.0,40,1,2021-12-06 19:56:44.753217
